In [ ]:
# Import all necessary libraries
import requests 
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

### Q1 Getting the embeddings model

In [60]:
# Get the model
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

# Define the question
user_question = "I just discovered the course. Can I still join it?"

# Create a vector
q_vector = embedding_model.encode(user_question)

# Checking vector length
print("The vector length is", len(q_vector))

# Printing the final answer
print("Answer 1: ", str(round(q_vector[0], 3)))

The vector length is 768
Answer 1:  0.078


In [29]:
# Debugging purposes
q_vector
q_vector.shape

(768,)

In [5]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
print("Original list contains: ", len(documents), "documents")

# Filtering the documents to include only questions for "machine-learning-zoomcamp"
filtered_documents = [doc for doc in documents if doc.get('course') == 'machine-learning-zoomcamp']
print("Filtered list contains:", len(filtered_documents), "documents")

Original list contains:  948 documents
Filtered list contains: 375 documents


### Q2 Creating the embeddings

In [43]:
# Create a list
embeddings = []

# Iterate over each document
for doc in filtered_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    # Debugging
    # print("Current doc:", qa_text)
    
    # Compute the embeddings and add to the list
    qa_text_vector = embedding_model.encode(qa_text)
    embeddings.append(qa_text_vector)

# Put all embeddings into a single matrix
X = np.array(embeddings)

# Check the shape
print("Answer 2:", X.shape)

Answer 2: (375, 768)


In [46]:
# Debugging purposes
print("First few document embeddings:")
for i in range(3):
    print(embeddings[i])

First few document embeddings:
[ 8.80591348e-02  1.55936647e-02  7.92557970e-02  2.52757613e-02
  7.55764544e-02 -3.90596464e-02 -4.13813964e-02  2.52917763e-02
  2.43241433e-02  3.62592144e-03 -7.28287734e-03 -3.28751132e-02
  6.12956211e-02 -5.71100190e-02  1.16774580e-02 -1.79441478e-02
  4.49206121e-02 -5.41606359e-02 -1.92257459e-03  1.48330042e-02
  7.91351032e-03 -3.43126692e-02  1.21481009e-02  1.27185853e-02
 -1.60091929e-02  7.12128868e-03  1.58389192e-02 -3.27517511e-04
  3.01296078e-03 -4.58917804e-02 -1.34767359e-03  2.01150030e-02
  2.33723950e-02 -4.28195437e-03 -3.09383497e-02  1.94305147e-03
 -1.88219883e-02  5.35157043e-03  1.29352175e-02 -3.23958918e-02
  3.10557149e-02  5.12980996e-03 -3.24484805e-04 -3.15542668e-02
  1.93377547e-02 -6.38601631e-02 -3.05380952e-02 -6.27885088e-02
 -1.74859874e-02  3.52948904e-02 -2.98070889e-02  7.45345727e-02
 -2.06182282e-02 -1.79156773e-02  1.86262894e-02  5.31014390e-02
  1.19286599e-02 -5.18543683e-02 -1.61919708e-03  3.5870622

In [45]:
unique_embeddings = np.unique(X, axis=0)
print("Number of unique document embeddings:", len(unique_embeddings))

Number of unique document embeddings: 375


### Q3 Search

In [48]:
# Checking that the question vector is normalised
normalised = q_vector.dot(q_vector)
normalised

0.9999999

In [49]:
# Checking that the dot product is computer correctly - debugging
for i in range(3):
    print(f"Dot product of document {i} and user question embedding:", np.dot(X[i], q_vector))

Dot product of document 0 and user question embedding: 0.28921726
Dot product of document 1 and user question embedding: 0.4350509
Dot product of document 2 and user question embedding: 0.22057262


In [58]:
scores = X.dot(q_vector)
# print(scores)
max_index = np.argmax(scores)
print("Answer 3:", str(round(scores[max_index], 2)))

Answer 3: 0.65


### Search function

In [52]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = filtered_documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(q_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4 Hit-rate for the search engine

In [53]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [65]:
# Evaluate the search function
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        query_text = q['question']
        v_query = embedding_model.encode(query_text)
        results = search_function(v_query)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return "Answer 4:" + str(round(hit_rate(relevance_total), 5))

# Calculate the hit rate
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

evaluate(ground_truth, search_engine.search)

  0%|          | 0/1830 [00:00<?, ?it/s]

'Answer 4:0.93989'

### Q5 Indexing with Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs
